# Introduction


## Short decription of the problem

For a given set of 3D polygons (e.g.: 3 or 4 3D polygons of arbitrary shapes and not necessarily standing in parallel planes), we want to compute the solid angle subtended by the intersection of these polygons as seen from a large number of arbitrary points in 3D space. The number of points is typically $~10^{5-6}$.

## Numerical strategy

The inputs are:
* A (3,Np) array of 3D (X,Y,Z) cartesian coordinates of Np points
* A list of (3,Nii) arrays of 3D (X,Y,Z) cartesian coordinates of closed polygons (polygon nb. ii has Nii-1 differen t points)
* An optional (3,) array of 3D (X,Y,Z) coordinates of a point P definign a plane on which to project all polygons
* An optional (3,) array of 33 (X,Y,Z) coordinates of a normalized vector nP defining, with P, a plane
* An optional (3,) array of 3D (X,Y,Z) coordinates of a normalized vector e1P defining a local (X1,X2) coordinate system in the plane defined by (P,nP)
* An optional (3,) array of 3D (X,Y,Z) coordinates of a normalized vector e2P defining a local (X1,X2) coordinate system in the plane defined by (P,nP)
* An optional list of (3,) arrays of 3D (X,Y,Z) coordinates of normalized vectors indicating the relevant side of each polygon (i.e.: the side looking towards the plasma)

Then, only the points standing on the "good" side of all polygons are considered.
For each of them, the projection of all polygons on a single plane is computed (homothetic), their intersection is computed. Using this intersection, we compute the solid angle from the point and the normalized vector oriented from the point towards the center of mass of the intersection. 

# Code

In [1]:
%load_ext cython

In [ ]:
%load_ext line_profiler
import line_profiler
import Cython.Compiler.Options as CCO

CCO._directive_defaults['linetrace'] = True
CCO._directive_defaults['binding'] = True

## Original version

In [2]:
%%cython -a
#-f --compile-args=-DCYTHON_TRACE=1
import numpy as np
cimport numpy as np
import math
import Polygon as plg
cimport cython
import datetime as dtm

ctypedef np.float_t DTYPE_t


cdef inline DTYPE_t Cabs(DTYPE_t x):
    return x if x >= 0.0 else -x

@cython.boundscheck(False)
@cython.wraparound(False)
cdef Calc_SAngVect_1Point1Poly_FromList(DTYPE_t P0, DTYPE_t P1, DTYPE_t P2, list Poly, DTYPE_t G0, DTYPE_t G1, DTYPE_t G2):
    # Return solid angle subtended by Poly from Point, as well as normalised vector from Points to center of mass of Poly
    cdef Py_ssize_t NPoly = len(Poly), ii = 0
    cdef DTYPE_t PG0 = G0-P0, PG1 = G1-P1, PG2 = G2-P2
    cdef DTYPE_t PGn = math.sqrt(PG0*PG0+PG1*PG1+PG2*PG2)
    cdef DTYPE_t PP10, PP11, PP12, PP20, PP21, PP22, PP1n, PP2n, ATan1, ATan2
    cdef DTYPE_t SAngs=0., V0=PG0/PGn, V1=PG1/PGn, V2=PG2/PGn
    for ii in xrange(0,NPoly-1):
        PP10, PP11, PP12 = Poly[ii][0]-P0, Poly[ii][1]-P1, Poly[ii][2]-P2
        PP20, PP21, PP22 = Poly[ii+1][0]-P0, Poly[ii+1][1]-P1, Poly[ii+1][2]-P2
        PP1n = math.sqrt(PP10*PP10+PP11*PP11+PP12*PP12)
        PP2n = math.sqrt(PP20*PP20+PP21*PP21+PP22*PP22)
        ATan1 = Cabs(PG0*(PP11*PP22-PP12*PP21) + PG1*(PP12*PP20-PP10*PP22) + PG2*(PP10*PP21-PP11*PP20))
        ATan2 = PGn*PP1n*PP2n + (PG0*PP10+PG1*PP11+PG2*PP12)*PP2n + (PG0*PP20+PG1*PP21+PG2*PP22)*PP1n + (PP10*PP20+PP11*PP21+PP12*PP22)*PGn
        sa = math.atan2(ATan1, ATan2)
        if sa>=0:
            SAngs = SAngs + sa
        else:
            SAngs = SAngs + sa + np.pi
    return 2.*SAngs, V0,V1,V2


@cython.boundscheck(False)
@cython.wraparound(False)
def Calc_SAngV_original(list LPolys, np.ndarray[DTYPE_t, ndim=2] Points, np.ndarray[DTYPE_t, ndim=1,mode='c'] P, np.ndarray[DTYPE_t, ndim=1,mode='c'] nP, np.ndarray[DTYPE_t, ndim=1,mode='c'] e1P, np.ndarray[DTYPE_t, ndim=1,mode='c'] e2P):
    # Returns homothetic (with centers As) projections of Poly on plane (P,nP), and optionnaly their components (X1,X2) along (e1P,e2P)
    # Arbitrary Points and plane, but a unique plane and polygon list common to all of them
    #t1, t2, t3, t4, t5, t6 = 0.,0.,0.,0.,0.,0.    # DB
    #tt = dtm.datetime.now()                        # DB
    cdef Py_ssize_t NP = Points.shape[1], NList = len(LPolys)
    cdef Py_ssize_t ii, jj, indOKii
    cdef DTYPE_t P0=P[0], P1=P[1], P2=P[2], e10=e1P[0], e11=e1P[1], e12=e1P[2], e20=e2P[0], e21=e2P[1], e22=e2P[2]
    cdef np.ndarray[DTYPE_t, ndim=1, mode='c']  SAng = np.zeros((NP,))
    cdef np.ndarray[DTYPE_t, ndim=2]  Vect = np.nan*np.ones((3,NP))
    cdef np.ndarray[np.int_t, ndim=1, mode='c'] NPperPoly = np.array([LPolys[ii].shape[1] for ii in range(0,NList)],dtype=int)
    cdef np.ndarray[DTYPE_t, ndim=2]  Polys = np.concatenate(tuple(LPolys),axis=1)
    cdef Py_ssize_t                   NPoly = Polys.shape[1]
    cdef np.ndarray[DTYPE_t, ndim=3]  Asbis = np.tile(Points.reshape((3,1,NP)),(1,NPoly,1)).T
    cdef np.ndarray[DTYPE_t, ndim=3]  AM = np.tile(Polys.T.reshape(1,NPoly,3),(NP,1,1)) - Asbis
    cdef np.ndarray[DTYPE_t, ndim=3]  Pbis = np.tile(P.reshape(1,1,3),(NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=3]  nPbis = np.tile(nP.reshape(1,1,3),(NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=3]  ScaAMn = np.sum(AM*nPbis,axis=2,keepdims=True)
    cdef np.ndarray[DTYPE_t, ndim=3]  ScaAPn = np.sum((Pbis-Asbis)*nPbis,axis=2,keepdims=True)
    cdef np.ndarray indneg = np.any(ScaAMn*ScaAPn<0,axis=1).flatten()
    cdef Py_ssize_t[::1]              IndOK = (~indneg).nonzero()[0]
    cdef np.ndarray[DTYPE_t, ndim=3]  k = np.zeros((NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=3]  e1bis = np.tile(e1P.reshape(1,1,3),(NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=3]  e2bis = np.tile(e2P.reshape(1,1,3),(NP,NPoly,1))
    cdef np.ndarray[DTYPE_t, ndim=2]  PolyProjX1, PolyProjX2
    cdef DTYPE_t Bs0, Bs1
    cdef list Polyint, PX, PolyInt
    cdef list IndPoly = [(np.sum(NPperPoly[0:jj+1])-NPperPoly[jj], np.sum(NPperPoly[0:jj+1])-1) for jj in range(0,NList)]
    #t1 = (dtm.datetime.now()-tt).total_seconds()      # DB
    #tt = dtm.datetime.now()         # DB
    # Get the intersection of projected polygons for each point
    ind = np.abs(ScaAMn)-1.e-14 > 0.
    k[ind] = ScaAPn[ind]/ScaAMn[ind]
    k[indneg,:,:] = np.nan
    cdef np.ndarray[DTYPE_t, ndim=3] PDiff = Asbis + (np.tile(k.T,(3,1,1)).T)*AM - Pbis # = PolyProj - Pbis
    PolyProjX1 = np.sum(PDiff*e1bis,axis=2,keepdims=False)
    PolyProjX2 = np.sum(PDiff*e2bis,axis=2,keepdims=False)
    PX = [np.array([PolyProjX1[:,IndPoly[jj][0]:IndPoly[jj][1]+1],PolyProjX2[:,IndPoly[jj][0]:IndPoly[jj][1]+1]]).T for jj in range(0,NList)]
    #t2 = (dtm.datetime.now() - tt).total_seconds()    # DB
    # Compute solid angle for each point with non-zero intersection
    #tt3 = dtm.datetime.now()         # DB
    for ii in xrange(0,IndOK.size):
        #tt = dtm.datetime.now()         # DB
        indOKii = IndOK[ii]
        Polyint = [plg.Polygon(PX[0][:,indOKii,:])]
        for jj in range(1,NList):
            Polyint[0] = Polyint[0] & plg.Polygon(PX[jj][:,indOKii,:])
        #t4 += (dtm.datetime.now()-tt).total_seconds()   # DB
        if Polyint[0].area()>1e-12:
            #tt = dtm.datetime.now()    # DB
            Bs0, Bs1 = Polyint[0].center()
            PolyInt = [(P0+e10*pp[0]+e20*pp[1],P1+e11*pp[0]+e21*pp[1],P2+e12*pp[0]+e22*pp[1]) for pp in Polyint[0][0]+[Polyint[0][0][0]]]
            #t5 += (dtm.datetime.now()-tt).total_seconds()   # DB
            #tt = dtm.datetime.now()    # DB
            SAng[indOKii], Vect[0,indOKii], Vect[1,indOKii], Vect[2,indOKii] = Calc_SAngVect_1Point1Poly_FromList(Points[0,indOKii],Points[1,indOKii],Points[2,indOKii], PolyInt, P0+e10*Bs0+e20*Bs1, P1+e11*Bs0+e21*Bs1, P2+e12*Bs0+e22*Bs1)
            #t6 += (dtm.datetime.now()-tt).total_seconds()   # DB
    #t3 = (dtm.datetime.now() - tt3).total_seconds()    # DB
    #print t1, t2, t3, t4, t5, t6 # DB
    return SAng, Vect, indneg

## Cython

In [3]:
%%cython -a
#-f --compile-args=-DCYTHON_TRACE=1
import numpy as np
cimport numpy as np
import Polygon as plg
cimport cython
from cpython cimport bool
from libc.math cimport abs as Cabs, sqrt as Csqrt
from libc.math cimport atan2 as Catan2, pi as Cpi

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Calc_SAngV_1Pt1Poly_cython(double P0, double P1, double P2, list Poly, double G0, double G1, double G2):
    # Return solid angle subtended by Poly from Point, as well as normalised vector from Points to center of mass of Poly
    cdef int NPoly = len(Poly), ii
    cdef double PG0 = G0-P0, PG1 = G1-P1, PG2 = G2-P2
    cdef double PGn = Csqrt(PG0*PG0+PG1*PG1+PG2*PG2)
    cdef double PP10, PP11, PP12, PP20, PP21, PP22, PP1n, PP2n, ATan1, ATan2
    cdef double SAngs=0., V0=PG0/PGn, V1=PG1/PGn, V2=PG2/PGn, sa
    for ii in range(0,NPoly-1):
        PP10, PP11, PP12 = Poly[ii][0]-P0, Poly[ii][1]-P1, Poly[ii][2]-P2
        PP20, PP21, PP22 = Poly[ii+1][0]-P0, Poly[ii+1][1]-P1, Poly[ii+1][2]-P2
        PP1n = Csqrt(PP10*PP10+PP11*PP11+PP12*PP12)
        PP2n = Csqrt(PP20*PP20+PP21*PP21+PP22*PP22)
        ATan1 = Cabs(PG0*(PP11*PP22-PP12*PP21) + PG1*(PP12*PP20-PP10*PP22) + PG2*(PP10*PP21-PP11*PP20))
        ATan2 = PGn*PP1n*PP2n + (PG0*PP10+PG1*PP11+PG2*PP12)*PP2n + (PG0*PP20+PG1*PP21+PG2*PP22)*PP1n + (PP10*PP20+PP11*PP21+PP12*PP22)*PGn
        sa = Catan2(ATan1, ATan2)
        if sa>=0:
            SAngs = SAngs + sa
        else:
            SAngs = SAngs + sa + Cpi
    return 2.*SAngs, V0,V1,V2
    

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
def Calc_SAngV_cython(list LPolys, double[:,::1] Pts, double[::1] P, double[::1] nP, double[::1] e1P, double[::1] e2P):
    # Returns homothetic (with centers As) projections of Poly on plane (P,nP), and optionnaly their components (X1,X2) along (e1P,e2P)
    # Arbitrary Points and plane, but a unique plane and polygon list common to all of them
    cdef int NP=Pts.shape[1], NL = len(LPolys)
    cdef int ii, jj, NT=0, nn=0, Nint
    cdef double scaAMn, scaAPn, area, Bs1, Bs2, C0, C1, C2, v0, v1, v2, v0b, v1b, v2b
    cdef long[::1] N
    cdef double[:,::1] LPxyz
    cdef list Pint2D, Pint3D
    cdef np.ndarray[double, ndim=2] LPx12
    cdef np.ndarray[double, ndim=1] SAng = np.zeros((NP,))
    cdef np.ndarray[double, ndim=2] Vect = np.nan*np.ones((3,NP))
    cdef np.ndarray[long, ndim=1]   indneg = np.zeros((NP,),dtype=int)
    
    # Pre-format the polygon coordinates for faster indexing
    N = np.empty((NL,),dtype=int)
    for ii in range(0,NL):
        N[ii] = LPolys[ii].shape[1]
        NT += N[ii]
    LPxyz = np.empty((3,NT))
    for ii in range(0,NL):
        for jj in range(0,N[ii]):
            LPxyz[0,nn+jj] = LPolys[ii][0,jj]
            LPxyz[1,nn+jj] = LPolys[ii][1,jj]
            LPxyz[2,nn+jj] = LPolys[ii][2,jj]
        nn += N[ii]
    # Loop on points
    LPx12 = np.empty((NT,2))
    for ii in range(0,NP):
        scaAPn = (P[0]-Pts[0,ii])*nP[0] + (P[1]-Pts[1,ii])*nP[1] + (P[2]-Pts[2,ii])*nP[2]
        for jj in range(0,NT):
            #v0, v1, v2 = LPxyz[0,jj]-Pts[0,ii], LPxyz[1,jj]-Pts[1,ii], LPxyz[2,jj]-Pts[2,ii]
            scaAMn = (LPxyz[0,jj]-Pts[0,ii])*nP[0] + (LPxyz[1,jj]-Pts[1,ii])*nP[1] + (LPxyz[2,jj]-Pts[2,ii])*nP[2]
            if scaAMn*scaAPn<=0.:
                indneg[ii] = 1
                break
            else:
                LPx12[jj,0] = (Pts[0,ii]+(LPxyz[0,jj]-Pts[0,ii])*scaAPn/scaAMn)*e1P[0] + (Pts[1,ii]+(LPxyz[1,jj]-Pts[1,ii])*scaAPn/scaAMn)*e1P[1] + (Pts[2,ii]+(LPxyz[2,jj]-Pts[2,ii])*scaAPn/scaAMn)*e1P[2]
                LPx12[jj,1] = (Pts[0,ii]+(LPxyz[0,jj]-Pts[0,ii])*scaAPn/scaAMn)*e2P[0] + (Pts[1,ii]+(LPxyz[1,jj]-Pts[1,ii])*scaAPn/scaAMn)*e2P[1] + (Pts[2,ii]+(LPxyz[2,jj]-Pts[2,ii])*scaAPn/scaAMn)*e2P[2]
        if indneg[ii]==0:
            nn = N[0]
            Pint2D = [plg.Polygon(LPx12[:N[0],:])]
            for jj in range(1,NL):
                Pint2D[0] = Pint2D[0] & plg.Polygon(LPx12[nn:nn+N[jj],:])
                nn += N[jj]
            if Pint2D[0].nPoints()>2:
                Bs1, Bs2 = Pint2D[0].center()
                Pint3D = [(P[0]+e1P[0]*pp[0]+e2P[0]*pp[1],P[1]+e1P[1]*pp[0]+e2P[1]*pp[1],P[2]+e1P[2]*pp[0]+e2P[2]*pp[1]) for pp in Pint2D[0][0]+[Pint2D[0][0][0]]]
                SAng[ii], Vect[0,ii], Vect[1,ii], Vect[2,ii] = Calc_SAngV_1Pt1Poly_cython(Pts[0,ii],Pts[1,ii],Pts[2,ii], Pint3D, P[0]+e1P[0]*Bs1+e2P[0]*Bs2, P[1]+e1P[1]*Bs1+e2P[1]*Bs2, P[2]+e1P[2]*Bs1+e2P[2]*Bs2)
                
    return SAng, Vect, indneg.astype(bool)


### Profiling

In [4]:
import numpy as np

# Polygons
P0 = [[10.,10.,10.,10.],[-1.,1.,1.,-1.],[-1.,-1.,1.,1.]]
P1 = [[11.,11.,11.],    [-1.,1.,0.],    [-1.,-1.,1.]]
P2 = [[12.,12.,12.],    [0.,1.,-1.],    [-1.,1.,1.]]
P3 = [[10.,10.,10.,10.],[-1.,1.,1.,-1.],[-1.,-1.,1.,1.]]
P0, P1, P2, P3 = [np.concatenate((pp,np.array(pp)[:,0:1]),axis=1) for pp in [P0,P1,P2,P3]]
LPoly = [P0,P1,P2,P3]

# Points
X, Y, Z = np.linspace(-10,15,100), np.linspace(-10,10,100), np.linspace(-10,10,100)
Xf = np.tile(X,Y.size*Z.size).flatten()
Yf = np.tile(np.repeat(Y,X.size).flatten(),Z.size).flatten()
Zf = np.repeat(Z,X.size*Y.size).flatten()
Pts = np.array([Xf,Yf,Zf])

P, nP = np.array([10.,0.,0.]), np.array([-1.,0.,0.])
e1, e2 = np.array([0.,1.,0.]),np.array([0.,0.,1.])


In [ ]:
NP = 10000
prof = line_profiler.LineProfiler(Calc_SAngV_original)
prof.runcall(Calc_SAngV_original, LPoly, np.ascontiguousarray(Pts[:,:NP]), P, nP, e1, e2)
prof.print_stats()

In [ ]:
NP = 10000
prof = line_profiler.LineProfiler(Calc_SAngV_cython)
prof.runcall(Calc_SAngV_cython, LPoly, np.ascontiguousarray(Pts[:,:NP]), P, nP, e1, e2)
prof.print_stats()

# Benchmark 

In [7]:
NP = 400000
pts = np.ascontiguousarray(Pts[:,:NP])
SAngo, Vecto, indno = Calc_SAngV_original(LPoly, pts, P, nP, e1, e2)
SAngc, Vectc, indnc = Calc_SAngV_cython(LPoly, pts, P, nP, e1, e2)

print "SAng", np.allclose(SAngo,SAngc, atol=1.e-12,rtol=0.,equal_nan=True)
print "Vect", np.allclose(Vecto,Vectc, atol=1.e-12,rtol=0.,equal_nan=True)
print "indneg", np.allclose(indno,indnc, atol=1.e-12,rtol=0.,equal_nan=True)

%timeit SAngo, Vecto, indno = Calc_SAngV_original(LPoly, pts, P, nP, e1, e2)
%timeit SAngn, Vectn, indnc = Calc_SAngV_cython(LPoly, pts, P, nP, e1, e2)



SAng True
Vect True
indneg True
1 loop, best of 3: 16.4 s per loop
1 loop, best of 3: 8.73 s per loop


In [ ]:
print SAngo
print SAngc